In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

Radial basis function interpolation

- you are given scattered points {x_i,f_i}
- you want to find a function s(x_i) = f_i that interpolates these x_i,f_i 
- this function shuld be smooth inbetween the {x_i,f_i}'s

we approximate s(x) as a weighted sum of radially symmetric kernels centered at each x_i
$$
s(x) = \Sigma_{i=1}^N \omega_i \phi(\Vert x - x_i \Vert)
$$

where $\omega_i$ are unknown weights and $\phi(r)$ is a __radial__ basis function.

To find the $\{ \omega_i\}$'s we ENFORCE 
$$ 
s(x_j) = \Sigma_{i=1}^N \omega_i \phi(\vert x_j - x_i\vert) = f_j
$$ 

This  gives a linear system $\mathbf{\Phi} \vec{\omega} = \vec{f}$ where $\Phi_{ij} = \phi(\vert x_i - x_j \vert)$. 

Then $\vec{\omega} = \mathbf{\Phi}^{-1} \vec{f}$

Once we have the _optimized_ $\vec{\omega}$ we can use that for evaluating the the values in-between the $\{x_i,f_i\}$s

questions:
- how do you cap the $N$?
- does s(x_j) have to be exactly equal to f_j or are we just happy with a minimization like we did with linear interpolation?


### Common RBF choices
- Gaussian: $\exp[-\epsilon r^2]$
- Multi-Quadratic: $\sqrt{1 + (\epsilon r)^2}$
- Inverse MQ: $1/\sqrt{1 + (\epsilon r)^2}$
- Thin Plate Spline : $r^2 \log(r)$

In [ ]:
np.random.randn(10)/100

In [ ]:
# 1D example
# we'll sample from f(x) = sin(x) 
# and then reconstruct the s(x)
# we'll use gaussian RBF

np.random.seed(0)
nSamples = 30
x = np.linspace(0, 2*np.pi, nSamples)
f = np.sin(x) + np.random.randn(nSamples)/1000

# Define RBF
def gaussian_rbf(r, epsilon=1.0):
    return np.exp(-(epsilon*r)**2)

# build the Phi Matrix
def rbf_interpolation_matrix(x,rbf,epsilon):
    N = len(x)
    Phi = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            r = abs(x[i] - x[j])
            Phi[i,j] = rbf(r,epsilon)
    return Phi

# compute weights
epsilon = 1.0
Phi = rbf_interpolation_matrix(x, gaussian_rbf, epsilon)
w = np.linalg.solve(Phi,f)

# we now haev the optimized weights
# so lets use that to plot the function eveywhere

nEvals = 200
x_eval = np.linspace(0,2*np.pi,nEvals)
Phi_eval = np.zeros((len(x_eval), len(x)))
for i,xe in enumerate(x_eval):
    r = abs(xe - x)
    Phi_eval[i,:] = gaussian_rbf(r,epsilon)

f_interp = Phi_eval @ w

plt.figure(figsize=(8,5))
plt.scatter(x, f, color='r', label='Samples')
plt.plot(x_eval, np.sin(x_eval), 'g--', label='True function')
plt.plot(x_eval, f_interp, 'b', label='RBF interpolation')
plt.legend()
plt.xlabel("x")
plt.ylabel("f(x)")
plt.title("RBF Interpolation using Gaussian kernel")
plt.show()


Question - will imposing boundary conditions make it more accurate at the edges? 

In [ ]:
# 2D version 
# f(x,y) = sin(x)*cos(y)

np.random.seed(0)
nSamples = 200
x = np.random.uniform(-2*np.pi, 2*np.pi, nSamples)
y = np.random.uniform(-2*np.pi, 2*np.pi, nSamples)
f = np.sin(x)*np.cos(y) #+ np.random.randn(nSamples,nSamples)/100

points = np.vstack([x,y]).T

# def pairwise_dist(A,B):
#     aElts,aDim = A.shape # number of elements, number of dimensions for each element
#     bElts,bDim = B.shape # number of elements, number of dimensions for each element
#     dist = np.zeros((aElts,bElts))
#     for i in range(aElts):
#         for j in range(bElts):
#             diff = A[i] - B[i]
#             dist[i,j] = np.sqrt(np.sum(diff**2)) # Euclidean norm
    
#     return dist

# A and B are matrices. Each row of the matrix is a point in N-dimension. in this example N = 2 just (x,y)
# efficient method for computing distances between all points in one go. numpy uses C code under the hood
def pairwise_dist(A, B):
    return np.linalg.norm(A[:,None,:] - B[None,:,:], axis=-1)

epsilon = 1.0
Phi = gaussian_rbf(pairwise_dist(points,points),epsilon)
w = np.linalg.solve(Phi,f)


# # Evaluate on a grid
nEvals = 100
xg = np.linspace(-2*np.pi, 2*np.pi, nEvals)
yg = np.linspace(-2*np.pi, 2*np.pi, nEvals)
Xg,Yg = np.meshgrid(xg,yg)
grid = np.stack([Xg.ravel(), Yg.ravel()],axis=1)

Phi_eval = gaussian_rbf(pairwise_dist(grid,points),epsilon)
f_interp = Phi_eval @ w
f_interp = f_interp.reshape(Xg.shape)

plt.figure(figsize=(6,5))
plt.contourf(Xg, Yg, f_interp, levels=30, cmap='viridis')
plt.scatter(x, y, c='r', s=10)
plt.title("2D RBF Interpolation")
plt.colorbar(label='f(x,y)')
plt.show()

# Create a 3D surface plot
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

# Plot the interpolated surface
surf = ax.plot_surface(Xg, Yg, f_interp, cmap='viridis', edgecolor='none')

# Optionally, overlay the original sample points
# ax.scatter(x, y, f_interp.flatten()[::len(f_interp)//len(x)], c='r', s=20, label='Sample Points')
ax.scatter(x, y, f, c='r', s=20, label='Sample Points')

# Labels and title
ax.set_title("3D RBF Interpolation Surface")
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('f(x, y)')

# Colorbar
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label='f(x,y)')

plt.show()


## 3D point cloud interpolation

- you are given a point cloud $\{ p_i\} \in \mathbb{R}^3$ which were sampled from a surface
- find a smooth function $f(\vec{x}) : \mathbb{R}^3 \rightarrow \mathbb{R}$ such that
    - $f(\vec{p_i}) = 0$ and
    - $f(x) > 0$ outside the surface and
    - $f(x) < 0$ inside the surface
- This is called a __zero level set__ $\{ x | f(x) = 0\}$

We represent f(x) as an RBF plus a polynomial. The polynomial is to help avoid trivial zero solutions or if the RBF matrix were singular
$$ f(\vec{x}) = \Sigma_{i=1}^{N} \omega_i \phi(\vert \vec{x} - \vec{p_i} \vert)  + \vec{a}\vec{x} + b$$ 
where $\vec{a}$ is a constant vector and $b$ is a constant. 

We need to evaluate $\vec{\omega}, \vec{a}, b $

### TRICK - Use offset points to generate more points. 
We have points on the surface $f(\vec{p_i}) = 0$. so to generate more points we can perturb them by a small $\delta$ along the normal to the surface to get more points:
- $\vec{p_i}^+ = \vec{p_i} + \delta.\hat{n}$ (outside the surfacE)
- $\vec{p_i}^- = \vec{p_i} - \delta.\hat{n}$ (inside the surfacE)


So now, for ecah point $\vec{q_i}$ (which includes originals and offsets), we get 
$$ \Sigma_i \omega_i \phi(\vert \vec{q}_i - \vec{p}_j\vert) + \vec{a}\vec{q} + b = f_j$$ 

Or in Block-Matrix form 

$$ 
\begin{bmatrix} 
\Phi & \vec{P} \\ \vec{P}^T & 0
\end{bmatrix} 
\begin{bmatrix} 
\vec{\omega} \\ \vec{a}
\end{bmatrix} = 
\begin{bmatrix} 
f \\ 0
\end{bmatrix} 
$$ 

where 
- $\Phi_{ij} = \phi(\vert \vec{q}_j - \vec{p}_i \vert )$
- $\vec{P} = [1, x_i, y_i, z_i]$

In [32]:
print(P.shape,Phi.shape)

(600, 4) (600, 200)


In [33]:
A = np.block([
    # [Phi, P],
    [P.T, np.zeros((4,4))]
])

B = np.block([
    [Phi, P],
    # [P.T, np.zeros((4,4))]
])

In [30]:
print(A.shape,B.shape)

(4, 604) (600, 204)


In [18]:
# 3D point cloud interpolation
# we'll sample points from a spehere 

np.random.seed(0)
N = 
phi = 2 * np.pi * np.random.rand(N)
costheta = 2*np.random.rand(N) - 1
theta = np.arccos(costheta)
x = np.sin(theta) * np.cos(phi)
y = np.sin(theta) * np.sin(phi)
z = np.cos(theta)
points = np.vstack([x, y, z]).T

#Normals 
normals = points.copy()

delta = 0.05
points_in = points - delta * normals
points_out = points + delta * normals

# combine all poitns
Q = np.vstack([
    points,              # original surface points 
    points_in,           # generated inside-points
    points_out           # generated outside-points
])
f = np.concatenate([
    np.zeros(N),          # surface
    -delta*np.ones(N),    # inside
    +delta*np.ones(N)     # outside
])

# Gaussian RBF
def gaussian_rbf(r, epsilon=2.0):
    return np.exp(-(epsilon * r)**2)

# Pairwise distance
def pairwise_dist(A, B):
    return np.linalg.norm(A[:, None, :] - B[None, :, :], axis=-1)

# build block matrix equation
epsilon = 2.0
Phi = gaussian_rbf(pairwise_dist(Q, points), epsilon)

# Polynomial terms
P = np.hstack([np.ones((len(Q),1)), Q])

A = np.block([
    [Phi, P],
    [P.T, np.zeros((4,4))]
])

b = np.concatenate([f,np.zeros(4)])
# solve
w_a = np.linalg.lstsq(A,b,rcond=None)[0]
w = w_a[:-4]
a = w_a[-4:]

# Evaluate on a grid
grid_x,grid_y,grid_z = [np.linspace(-1.5,1.5,40)]*3
X,Y,Z = np.meshgrid(grid_x,grid_y,grid_z)
grid = np.stack([X.ravel(), Y.ravel(), Z.ravel()], axis=1)

Phi_eval = gaussian_rbf(pairwise_dist(grid, points), epsilon)
P_eval = np.hstack([np.ones((len(grid),1)), grid])
f_eval = Phi_eval @ w + P_eval @ a
f_eval = f_eval.reshape(X.shape)

# Visualize implicit surface (f=0)
from skimage import measure
verts, faces, normals, values = measure.marching_cubes(f_eval, level=0, spacing=(grid_x[1]-grid_x[0],)*3)

# Plot
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')
ax.plot_trisurf(verts[:,0]-1.5, verts[:,1]-1.5, faces, verts[:,2]-1.5,
                color='lightblue', alpha=0.6, linewidth=0)
ax.scatter(points[:,0], points[:,1], points[:,2], color='r', s=10)
ax.set_title("RBF Surface Reconstruction from Point Cloud")
plt.show()


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 204 and the array at index 1 has size 604

In [16]:
np.concatenate([Phi.T,P.T])

array([[ 1.00000000e+00,  3.13988507e-03,  6.15067646e-01, ...,
         7.05264878e-05,  6.61708275e-04,  1.54308015e-01],
       [ 3.13988507e-03,  1.00000000e+00,  3.53968952e-02, ...,
         6.60044890e-08,  3.11464135e-06,  8.80757244e-06],
       [ 6.15067646e-01,  3.53968952e-02,  1.00000000e+00, ...,
         4.35264158e-06,  2.23338607e-04,  1.81176595e-02],
       ...,
       [-8.83219805e-01, -1.99552236e-01, -7.74449899e-01, ...,
        -2.75322072e-02,  2.48649139e-01, -7.47530340e-01],
       [-2.79713454e-01, -8.97761573e-01, -5.83479993e-01, ...,
         1.00558034e+00,  9.49026255e-02,  3.26743588e-01],
       [-3.76408236e-01,  3.92686978e-01, -2.44496321e-01, ...,
        -3.00915530e-01, -1.01571014e+00, -6.61012117e-01]],
      shape=(204, 600))